In [25]:
import pandas as pd
df = pd.read_csv("raw_data/ustc_historic_data.csv", sep=";")
df.head()
df.columns

Index(['timeOpen', 'timeClose', 'timeHigh', 'timeLow', 'name', 'open', 'high',
       'low', 'close', 'volume', 'marketCap', 'circulatingSupply',
       'timestamp'],
      dtype='object')

In [26]:
#clean datetime predictors
time_cols = ["timeOpen", "timeClose", "timeHigh", "timeLow", "timestamp"]
df[time_cols] = df[time_cols].apply(pd.to_datetime)

#standardise date reference 
df["date"] = df["timestamp"].dt.normalize()

df = df.drop(columns=time_cols)


In [27]:
#setting up predictors
import numpy as np

# sort and index date
df = df.sort_values('date').set_index('date')
df = df.asfreq('D', method='ffill') 

# 1. PRICE DEVIATION FROM $1
df['price_dev'] = df['close'] - 1
df['abs_price_dev'] = abs(df['price_dev'])
df['rel_price_dev'] = df['price_dev'] / 1

In [28]:
# 2. PRICE VOLATILITY / DAILY SWINGS
df['intraday_range_pct'] = (df['high'] - df['low']) / df['close']  # daily swing
df['volatility_7d'] = df['close'].pct_change().rolling(7).std()     # rolling 7-day volatility

In [29]:
# 3. VOLUME / LIQUIDITY STRESS
df['volume_zscore'] = (df['volume'] - df['volume'].rolling(7).mean()) / df['volume'].rolling(20).std()
df['volume_acceleration'] = df['volume_zscore'].diff().rolling(3).mean()
df['turnover'] = df['volume'] / df['circulatingSupply']
df['range_expansion'] = df['intraday_range_pct'] / df['intraday_range_pct'].rolling(7).mean()
# Optional: df['volume_chg'] = df['volume'].pct_change()
# Optional: df['volume_ma7'] = df['volume'].rolling(7).mean()
# Optional: df['price_range_proxy'] = df['intraday_range_pct']

In [30]:
# 4. SUPPLY & MARKET CAP CHANGES
df['supply_chg'] = df['circulatingSupply'].pct_change()
df['mcap_chg'] = df['marketCap'].pct_change()
df['mcap_velocity'] = df['marketCap'].pct_change(3)
df['supply_panic'] = (df['circulatingSupply'].pct_change().abs() > 0.05)
df['volume_mcap_ratio'] = df['volume'] / df['marketCap']

In [31]:
#5. EARLY WARNING / ACCELERATION SIGNALS
df['dev_acceleration'] = df['abs_price_dev'].diff().rolling(3).mean()
df['peg_stable_days'] = (df['close'] > 0.999).astype(int).groupby(
    (df['close'] <= 0.999).astype(int).cumsum()
).cumsum()
df['peg_breakdown'] = df['peg_stable_days'].diff().fillna(0)
df['price_shock'] = (abs(df['close'].pct_change()) > 0.02).astype(int)
df['volume_explosion'] = (df['volume'] > df['volume'].rolling(30).quantile(0.95)).astype(int)


In [21]:
df.head()



,name,open,high,low,close,volume,marketCap,circulatingSupply,price_dev,abs_price_dev,...,intraday_range_pct,volatility_7d,volume_chg,volume_ma7,volume_zscore,supply_chg,mcap_chg,volume_mcap_ratio,price_range_proxy,turnover
date,,,,,,,,,,,,,,,,,,,,,
2021-12-28 00:00:00+00:00,2781,1.000560,1.003111,0.999108,1.002239,1.395190e+08,1.005642e+10,1.003396e+10,0.002239,0.002239,...,0.003994,NaN,NaN,NaN,NaN,NaN,NaN,0.013874,0.003994,0.013905
2021-12-29 00:00:00+00:00,2781,1.002244,1.003347,0.999270,1.001423,1.575926e+08,1.007944e+10,1.006511e+10,0.001423,0.001423,...,0.004072,NaN,0.129542,NaN,NaN,0.003105,0.002289,0.015635,0.004072,0.015657
2021-12-30 00:00:00+00:00,2781,1.001019,1.002950,0.999730,1.001984,1.106219e+08,1.011771e+10,1.009768e+10,0.001984,0.001984,...,0.003213,NaN,-0.298051,NaN,NaN,0.003235,0.003797,0.010933,0.003213,0.010955
2021-12-31 00:00:00+00:00,2781,1.001748,1.002327,0.998924,1.001440,1.357036e+08,1.013712e+10,1.012254e+10,0.001440,0.001440,...,0.003398,NaN,0.226733,NaN,NaN,0.002463,0.001918,0.013387,0.003398,0.013406
2022-01-01 00:00:00+00:00,2781,1.001657,1.002204,0.998407,1.000757,1.266358e+08,1.013354e+10,1.012587e+10,0.000757,0.000757,...,0.003795,NaN,-0.066821,NaN,NaN,0.000329,-0.000353,0.012497,0.003795,0.012506
